# CatBoost Training Notebook
## Data Loading and Preprocessing for Recommendation System

This notebook loads click/non-click data, merges with creative features, and applies specific filtering rules to prepare a clean dataset for CatBoost model training.


In [20]:
import pandas as pd
import numpy as np
import warnings
from IPython.display import display, HTML

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")


✅ Libraries imported successfully!
📊 Pandas version: 2.3.3
🔢 NumPy version: 2.3.4


## 2. Load Data
Loading click/non-click variation data and creative features


In [31]:
# Load clicked and non-clicked datasets
print("📁 Loading datasets...")
print("="*60)

# Load clicked variations
clicked_df = pd.read_csv('data/clicked_variation_rows.csv')
clicked_df['CLICKED'] = 1
print(f"✅ Loaded clicked data: {clicked_df.shape}")

# Load non-clicked variations
non_clicked_df = pd.read_csv('data/non_clicked_variation_rows.csv')
non_clicked_df['CLICKED'] = 0
# Remove RN column if it exists (as seen in EDA notebook)
if 'RN' in non_clicked_df.columns:
    non_clicked_df = non_clicked_df.drop(columns=['RN'])
print(f"❌ Loaded non-clicked data: {non_clicked_df.shape}")

# Combine clicked and non-clicked data
combined_df = pd.concat([clicked_df, non_clicked_df], ignore_index=True)
print(f"\n🔄 Combined dataset shape: {combined_df.shape}")
print(f"📊 Total records: {len(combined_df):,}")
print(f"✅ Click rate: {combined_df['CLICKED'].mean():.2%}")
print("="*60)


📁 Loading datasets...
✅ Loaded clicked data: (47710, 24)
❌ Loaded non-clicked data: (47700, 24)

🔄 Combined dataset shape: (95410, 24)
📊 Total records: 95,410
✅ Click rate: 50.01%


In [32]:
# Load creative features
print("\n📎 Loading creative features...")
creative_features_df = pd.read_csv('data/combined_feats.csv', index_col=0)
print(f"Shape: {creative_features_df.shape}")
print(f"Unique variations: {creative_features_df['variation_id'].nunique()}")
print(f"Columns: {list(creative_features_df.columns)}")

# Join creative features with combined data
print("\n🔗 Merging with creative features...")
df = combined_df.merge(
    creative_features_df,
    left_on='VARIATION_ID',
    right_on='variation_id',
    how='left'
)

print(f"✅ Merged dataset shape: {df.shape}")
print(f"📊 Total records: {len(df):,}")

# Check for unmatched variations
unmatched = df['variation_id'].isna().sum()
print(f"\n⚠️ Unmatched variations (no creative features): {unmatched:,} ({unmatched/len(df):.2%})")



📎 Loading creative features...
Shape: (286, 15)
Unique variations: 286
Columns: ['variation_id', 'image_name', 'experiment_id', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'subject_line', 'category']

🔗 Merging with creative features...
✅ Merged dataset shape: (95410, 39)
📊 Total records: 95,410

⚠️ Unmatched variations (no creative features): 654 (0.69%)


## 3. Apply Data Filtering Rules
Applying sequential filtering to clean and prepare the dataset


In [33]:
print("🧹 Starting data filtering process...")
print("="*60)
initial_size = len(df)
print(f"Initial dataset size: {initial_size:,} rows\n")


🧹 Starting data filtering process...
Initial dataset size: 95,410 rows



In [34]:
# 1. Remove rows with null creative features
print("1️⃣ Removing rows with null creative features...")
before = len(df)
df = df[df['variation_id'].notna()]
after = len(df)
print(f"   Removed: {before - after:,} rows ({(before - after)/before:.2%})")
print(f"   Remaining: {after:,} rows\n")


1️⃣ Removing rows with null creative features...
   Removed: 654 rows (0.69%)
   Remaining: 94,756 rows



In [35]:
# 2. Geographic Filtering
print("2️⃣ Geographic filtering (keeping only United States)...")
before = len(df)

# First, convert 'US' to 'United States'
df['COUNTRY'] = df['COUNTRY'].replace('US', 'United States')

# Keep only United States
df = df[df['COUNTRY'] == 'United States']
after = len(df)
print(f"   Removed: {before - after:,} rows ({(before - after)/before:.2%})")
print(f"   Remaining: {after:,} rows\n")


2️⃣ Geographic filtering (keeping only United States)...
   Removed: 3,088 rows (3.26%)
   Remaining: 91,668 rows



In [36]:
# 3. Client Type Filtering
print("3️⃣ Client type filtering (removing library/email)...")
before = len(df)

# Remove library and email client types
df = df[~df['LATEST_CLICK_CLIENT_TYPE'].str.lower().str.contains('library|email', na=False)]
after = len(df)
print(f"   Removed: {before - after:,} rows ({(before - after)/before:.2%})")
print(f"   Remaining: {after:,} rows\n")


3️⃣ Client type filtering (removing library/email)...
   Removed: 27 rows (0.03%)
   Remaining: 91,641 rows



In [37]:
# 4. Client Name Filtering
print("4️⃣ Client name filtering (removing android/opera)...")
before = len(df)

# Remove android and opera client names
df = df[~df['LATEST_CLICK_CLIENT_NAME'].str.lower().str.contains('android|opera', na=False)]
after = len(df)
print(f"   Removed: {before - after:,} rows ({(before - after)/before:.2%})")
print(f"   Remaining: {after:,} rows\n")


4️⃣ Client name filtering (removing android/opera)...
   Removed: 89 rows (0.10%)
   Remaining: 91,552 rows



In [38]:
# 5. UTM Source Filtering
print("5️⃣ UTM source filtering (allowing explicit allowed sources + NaN)...")
before = len(df)

# Define allowed UTM sources
allowed_utm_sources = ['klaviyo', 'fb', 'google', 'ig', 'smsbump', 'applovin', 'arp', 'bing', 'facebook', 'orderlyemails']

# Make mask for allowed sources (case-insensitive) or NaN
utm_source_lower = df['FIRST_UTM_SOURCE'].str.lower()
allowed_mask = utm_source_lower.isin([s.lower() for s in allowed_utm_sources])
nan_mask = df['FIRST_UTM_SOURCE'].isna()
df = df[allowed_mask | nan_mask]

after = len(df)
print(f"   Allowed sources: {allowed_utm_sources} + NaN")
print(f"   Removed: {before - after:,} rows ({(before - after)/before:.2%})")
print(f"   Remaining: {after:,} rows\n")


5️⃣ UTM source filtering (allowing explicit allowed sources + NaN)...
   Allowed sources: ['klaviyo', 'fb', 'google', 'ig', 'smsbump', 'applovin', 'arp', 'bing', 'facebook', 'orderlyemails'] + NaN
   Removed: 522 rows (0.57%)
   Remaining: 91,030 rows



In [39]:
# 6. Experiment Size Filtering
print("6️⃣ Experiment size filtering (removing 5 experiments with <500 clicks)...")
before = len(df)

# Calculate clicks per experiment
experiment_clicks = df.groupby('EXPERIMENT_ID')['CLICKED'].sum().reset_index()
experiment_clicks.columns = ['EXPERIMENT_ID', 'total_clicks']

# Find experiments with less than 500 clicks
low_click_experiments = experiment_clicks[experiment_clicks['total_clicks'] < 500]
low_click_experiments = low_click_experiments.nsmallest(5, 'total_clicks')

print(f"   Experiments to remove (click counts): {low_click_experiments['total_clicks'].tolist()}")
print(f"   Experiment IDs: {low_click_experiments['EXPERIMENT_ID'].tolist()[:3]}...") # Show first 3 IDs

# Remove these experiments
df = df[~df['EXPERIMENT_ID'].isin(low_click_experiments['EXPERIMENT_ID'])]
after = len(df)
print(f"   Removed: {before - after:,} rows ({(before - after)/before:.2%})")
print(f"   Remaining: {after:,} rows\n")


6️⃣ Experiment size filtering (removing 5 experiments with <500 clicks)...
   Experiments to remove (click counts): [338]
   Experiment IDs: ['e627d7f0-46c4-4894-872e-59a2fc108c30']...
   Removed: 684 rows (0.75%)
   Remaining: 90,346 rows



In [40]:
# 7. Creative Feature Filtering
print("7️⃣ Creative feature filtering...")
before = len(df)

# Define filters for each creative feature
creative_filters = [
    ('Q3', 'aesthetic appeal', 'Messaging Approach'),
    ('Q7', 'three', 'Model Count'),
    ('Q8', '46-65', 'Age Group')
]

for col, value, description in creative_filters:
    before_filter = len(df)
    # Case-insensitive comparison
    df = df[df[col].str.lower() != value.lower()]
    removed = before_filter - len(df)
    if removed > 0:
        print(f"   {description} ({col}): removed '{value}' - {removed:,} rows")

after = len(df)
print(f"\n   Total removed: {before - after:,} rows ({(before - after)/before:.2%})")
print(f"   Remaining: {after:,} rows\n")


7️⃣ Creative feature filtering...
   Messaging Approach (Q3): removed 'aesthetic appeal' - 187 rows

   Total removed: 187 rows (0.21%)
   Remaining: 90,159 rows



In [99]:
# Convert FIRST_ACTIVE_TS to days from today, replace with numeric

# Today's date (corresponds to environment's current date)
from datetime import datetime
today = datetime(2025, 10, 17)

def convert_to_days_from_today(ts_str):
    try:
        if pd.isna(ts_str) or ts_str == 'missing':
            return np.nan
        # Parse the timestamp string
        ts = datetime.strptime(ts_str, '%Y-%m-%d %H:%M:%S.%f')
        days_diff = (today - ts).days
        return days_diff
    except:
        return np.nan

if 'FIRST_ACTIVE_TS' in df.columns:
    df['FIRST_ACTIVE_TS_DAYS'] = df['FIRST_ACTIVE_TS'].apply(convert_to_days_from_today)
    df = df.drop(columns=['FIRST_ACTIVE_TS'])

    print("✓ Converted FIRST_ACTIVE_TS to FIRST_ACTIVE_TS_DAYS (days from 2025-10-17)")
    print(f"Sample values:\n{df[['FIRST_ACTIVE_TS_DAYS']].head()}")



✓ Converted FIRST_ACTIVE_TS to FIRST_ACTIVE_TS_DAYS (days from 2025-10-17)
Sample values:
   FIRST_ACTIVE_TS_DAYS
0                282.00
2                102.00
3                108.00
4                 28.00
5                257.00


In [100]:
# 8. Remove columns
print("8️⃣ Removing  columns...")
columns_to_drop = ['LAST_EVENT_DATE', 'LATEST_CLICK_DATE', 'subject_line', 'image_name', 'variation_id', 'CLICK_COUNT', 'experiment_id']
existing_columns = [col for col in columns_to_drop if col in df.columns]

if existing_columns:
    df = df.drop(columns=existing_columns)
    print(f"   Dropped columns: {existing_columns}")
else:
    print(f"   No timestamp columns found to drop")
    
print(f"\n✅ Filtering complete!")
print(f"\n Dataset columns: {df.columns.tolist()}")

8️⃣ Removing  columns...
   No timestamp columns found to drop

✅ Filtering complete!

 Dataset columns: ['EXPERIMENT_ID', 'VARIATION_ID', 'MONTH', 'DAY', 'HOUR', 'RECIPIENT_ID', 'CITY', 'COUNTRY', 'REGION', 'TIMEZONE', 'LAST_ORDER_VALUE', 'TOTAL_ORDERS_VALUE', 'AVG_ORDER_VALUE', 'LATEST_CLICK_CLIENT_TYPE', 'LATEST_CLICK_CLIENT_OS_FAMILY', 'LATEST_CLICK_CLIENT_NAME', 'FIRST_UTM_SOURCE', 'FIRST_UTM_CONTENT', 'FIRST_UTM_CAMPAIGN', 'CLICKED', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'category', 'FIRST_ACTIVE_TS_DAYS']


## 4. Data Summary & Statistics


In [101]:
# Final dataset summary
print("📊 FINAL DATASET SUMMARY")
print("="*60)
print(f"\n📈 Size Statistics:")
print(f"   Initial dataset: {initial_size:,} rows")
print(f"   Final dataset: {len(df):,} rows")
print(f"   Total removed: {initial_size - len(df):,} rows ({(initial_size - len(df))/initial_size:.2%})")
print(f"   Retention rate: {len(df)/initial_size:.2%}")

print(f"\n✅ Click Statistics:")
print(f"   Total clicks: {df['CLICKED'].sum():,}")
print(f"   Total non-clicks: {(df['CLICKED'] == 0).sum():,}")
print(f"   Click rate: {df['CLICKED'].mean():.2%}")

print(f"\n🔬 Experiment Statistics:")
print(f"   Unique experiments: {df['EXPERIMENT_ID'].nunique()}")
print(f"   Unique variations: {df['VARIATION_ID'].nunique()}")
print(f"   Avg records per experiment: {len(df)/df['EXPERIMENT_ID'].nunique():.0f}")


📊 FINAL DATASET SUMMARY

📈 Size Statistics:
   Initial dataset: 95,410 rows
   Final dataset: 90,159 rows
   Total removed: 5,251 rows (5.50%)
   Retention rate: 94.50%

✅ Click Statistics:
   Total clicks: 45,097
   Total non-clicks: 45,062
   Click rate: 50.02%

🔬 Experiment Statistics:
   Unique experiments: 25
   Unique variations: 121
   Avg records per experiment: 3606


In [102]:
# Display data shape and columns
print("\n📊 Final Dataset Shape:")
print(f"   Rows: {df.shape[0]:,}")
print(f"   Columns: {df.shape[1]}")

print("\n📋 Column List:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i:2d}. {col}")
    
# Show sample of the final dataset
print("\n🔍 Sample of Final Dataset (first 5 rows):")
display(df.head())



📊 Final Dataset Shape:
   Rows: 90,159
   Columns: 32

📋 Column List:
    1. EXPERIMENT_ID
    2. VARIATION_ID
    3. MONTH
    4. DAY
    5. HOUR
    6. RECIPIENT_ID
    7. CITY
    8. COUNTRY
    9. REGION
   10. TIMEZONE
   11. LAST_ORDER_VALUE
   12. TOTAL_ORDERS_VALUE
   13. AVG_ORDER_VALUE
   14. LATEST_CLICK_CLIENT_TYPE
   15. LATEST_CLICK_CLIENT_OS_FAMILY
   16. LATEST_CLICK_CLIENT_NAME
   17. FIRST_UTM_SOURCE
   18. FIRST_UTM_CONTENT
   19. FIRST_UTM_CAMPAIGN
   20. CLICKED
   21. Q1
   22. Q2
   23. Q3
   24. Q4
   25. Q5
   26. Q6
   27. Q7
   28. Q8
   29. Q9
   30. Q10
   31. category
   32. FIRST_ACTIVE_TS_DAYS

🔍 Sample of Final Dataset (first 5 rows):


,EXPERIMENT_ID,VARIATION_ID,MONTH,DAY,HOUR,RECIPIENT_ID,CITY,COUNTRY,REGION,TIMEZONE,LAST_ORDER_VALUE,TOTAL_ORDERS_VALUE,AVG_ORDER_VALUE,LATEST_CLICK_CLIENT_TYPE,LATEST_CLICK_CLIENT_OS_FAMILY,LATEST_CLICK_CLIENT_NAME,FIRST_UTM_SOURCE,FIRST_UTM_CONTENT,FIRST_UTM_CAMPAIGN,CLICKED,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,category,FIRST_ACTIVE_TS_DAYS
0,002deaf7-331f-4b5e-866b-f6dad60e4a79,148d62bc-044f-4c55-9195-8be2d7579d9d,7,28,12,01HNP4BCBMFV98J5B420PCS38F,NaN,United States,NaN,America/Chicago,NaN,NaN,NaN,Mobile Browser,iOS,Mobile Safari,applovin,NaN,Underoutfit_CPP,1,Text,Lifestyle,Promotional / Incentive,Aesthetic Appeal,Bra,Studio,Two,26-35,Curvy,Unknown,Sensory / lifestyle,282.00
2,e4b4a349-3b14-439e-946f-f716101dac69,15be91b7-6b55-4cd3-8d07-f4068b018a3a,8,2,12,01HNP8Y1JFNW4G5YV6C5Z0X4SZ,Akron,United States,Ohio,America/New_York,177.00,177.00,177.00,Mobile Browser,Android,Chrome Mobile,Klaviyo,NaN,em - just dropped 460 459 457 - Sun Jul 6 20...,1,Model,Lifestyle,Social Proof / Validation,Unknown,Bra,Outdoor / Nature,One,26-35,Slim,White,Conversational tone,102.00
3,49c33d7c-ef04-43a7-bbd0-783489c64849,2dd79b7f-a399-459e-8eb0-6baf3a22f53b,9,6,13,01HNPADZ4935GQ7DA0Q76Z7J4V,Suffolk,United States,Virginia,America/New_York,NaN,NaN,NaN,Browser,Windows,Chrome,Klaviyo,NaN,SMS Group B - Buy more save more - Mon Jun 30 ...,1,Model,Lifestyle,Social Proof / Validation,Fit & Support,Bra,Bedroom / Home,One,18-25,Slim,White,Sensory / lifestyle,108.00
4,11c49e5d-21ac-4d6d-88c3-f211562a8e07,473c90cc-b034-4585-897c-da6900a9fa73,9,17,14,01HNPBGKJ35594G1B42RZ40RRV,Frontenac,United States,Missouri,America/Chicago,NaN,NaN,NaN,Mobile Browser,iOS,Mobile Safari,NaN,NaN,NaN,1,Model,Studio,Emotional / Lifestyle,Seamlessness,Bra & Underwear,Studio,One,26-35,Slim,White,Sensory / lifestyle,28.00
5,823158da-7b0a-4c19-8189-663c22a3ae38,84235e36-13c5-416d-acc0-1f3d7b1c9daf,9,27,12,01HNPF51YK5QZWG06XENZY6MDX,Carmel,United States,Indiana,America/Indiana/Indianapolis,NaN,NaN,NaN,Browser,OS X,Safari,Klaviyo,NaN,em - V7B for Valentines VDay - Fri Jan 31 2025...,1,Model,Studio,Functional / Product-Focused,Aesthetic Appeal,Bra,Studio,One,26-35,Curvy,White,Sensory / lifestyle,257.00


In [103]:
# Split data into train, validation, and test by saving last 2 experiments as test,
# 2 before those as validation, and all others as train,
# where experiments are sorted by their first exposure (MONTH, DAY) combo.

# Get the (MONTH, DAY) of first exposure for each experiment
exp_timing = (
    df.groupby('EXPERIMENT_ID')[['MONTH', 'DAY']]
    .min()
    .reset_index()
)

# Sort experiments by (MONTH, DAY)
exp_timing_sorted = exp_timing.sort_values(['MONTH', 'DAY'], ascending=[True, True])
sorted_experiment_ids = exp_timing_sorted['EXPERIMENT_ID'].tolist()

# Defining splits
test_experiments = sorted_experiment_ids[-2:]
val_experiments = sorted_experiment_ids[-4:-2]
train_experiments = sorted_experiment_ids[:-4]

# Gather info for printing
test_exp_info = exp_timing_sorted[exp_timing_sorted['EXPERIMENT_ID'].isin(test_experiments)]
val_exp_info = exp_timing_sorted[exp_timing_sorted['EXPERIMENT_ID'].isin(val_experiments)]

# Create DataFrames for each split
df_test = df[df['EXPERIMENT_ID'].isin(test_experiments)].copy()
df_val = df[df['EXPERIMENT_ID'].isin(val_experiments)].copy()
df_train = df[df['EXPERIMENT_ID'].isin(train_experiments)].copy()

print(f"\n🔀 Data Split (train / validation / test) based on experiment exposure order:")
print(f"   Train rows: {df_train.shape[0]:,}")
print(f"   Validation rows: {df_val.shape[0]:,}")
print(f"   Test rows: {df_test.shape[0]:,}")
print(f"   Train experiments: {df_train['EXPERIMENT_ID'].nunique()}")
print(f"   Validation experiments: {df_val['EXPERIMENT_ID'].nunique()}")
print(f"   Test experiments: {df_test['EXPERIMENT_ID'].nunique()}")
print(f"   Validation experiment IDs: {val_experiments}")
print(f"   Test experiment IDs: {test_experiments}")

# Print the (MONTH, DAY) of val and test experiments
print("\n🗓️  Dates of test experiments (by earliest exposure):")
for _, row in test_exp_info.iterrows():
    print(f"   TEST  EXPERIMENT_ID: {row['EXPERIMENT_ID']}  |  MONTH: {row['MONTH']}, DAY: {row['DAY']}")

print("\n🗓️  Dates of validation experiments (by earliest exposure):")
for _, row in val_exp_info.iterrows():
    print(f"   VAL   EXPERIMENT_ID: {row['EXPERIMENT_ID']}  |  MONTH: {row['MONTH']}, DAY: {row['DAY']}")




🔀 Data Split (train / validation / test) based on experiment exposure order:
   Train rows: 76,818
   Validation rows: 8,641
   Test rows: 4,700
   Train experiments: 21
   Validation experiments: 2
   Test experiments: 2
   Validation experiment IDs: ['823158da-7b0a-4c19-8189-663c22a3ae38', 'cd4a656f-290a-41e2-be1d-bf62ad85757d']
   Test experiment IDs: ['78a802ae-d6cd-4f39-aecb-138668fa2607', '0ef6d2e9-7601-4df6-a215-83e6e79aa24e']

🗓️  Dates of test experiments (by earliest exposure):
   TEST  EXPERIMENT_ID: 78a802ae-d6cd-4f39-aecb-138668fa2607  |  MONTH: 10, DAY: 2
   TEST  EXPERIMENT_ID: 0ef6d2e9-7601-4df6-a215-83e6e79aa24e  |  MONTH: 10, DAY: 6

🗓️  Dates of validation experiments (by earliest exposure):
   VAL   EXPERIMENT_ID: 823158da-7b0a-4c19-8189-663c22a3ae38  |  MONTH: 9, DAY: 27
   VAL   EXPERIMENT_ID: cd4a656f-290a-41e2-be1d-bf62ad85757d  |  MONTH: 9, DAY: 29


In [105]:
# Split df_train and df_test into features (X) and targets (y)
TARGET_COL = 'CLICKED'

X_train = df_train.drop(columns=[TARGET_COL])
y_train = df_train[TARGET_COL]

X_val = df_val.drop(columns=[TARGET_COL])
y_val = df_val[TARGET_COL]

X_test = df_test.drop(columns=[TARGET_COL])
y_test = df_test[TARGET_COL]
print(f"\n🧮 X_train shape: {X_train.shape}")
print(f"🧮 y_train shape: {y_train.shape}")
print(f"🧮 X_val shape: {X_val.shape}")
print(f"🧮 y_val shape: {y_val.shape}")
print(f"🧮 X_test shape:  {X_test.shape}")
print(f"🧮 y_test shape:  {y_test.shape}")

print("\n🔹 X_train columns:", list(X_train.columns))



🧮 X_train shape: (76818, 31)
🧮 y_train shape: (76818,)
🧮 X_val shape: (8641, 31)
🧮 y_val shape: (8641,)
🧮 X_test shape:  (4700, 31)
🧮 y_test shape:  (4700,)

🔹 X_train columns: ['EXPERIMENT_ID', 'VARIATION_ID', 'MONTH', 'DAY', 'HOUR', 'RECIPIENT_ID', 'CITY', 'COUNTRY', 'REGION', 'TIMEZONE', 'LAST_ORDER_VALUE', 'TOTAL_ORDERS_VALUE', 'AVG_ORDER_VALUE', 'LATEST_CLICK_CLIENT_TYPE', 'LATEST_CLICK_CLIENT_OS_FAMILY', 'LATEST_CLICK_CLIENT_NAME', 'FIRST_UTM_SOURCE', 'FIRST_UTM_CONTENT', 'FIRST_UTM_CAMPAIGN', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'category', 'FIRST_ACTIVE_TS_DAYS']


In [106]:
# First, let's identify which columns are recipient features vs variation features
# Recipient features: everything that's about the user
recipient_cols = [
    'EXPERIMENT_ID', 'RECIPIENT_ID', 'MONTH', 'DAY', 'HOUR',
    'CITY', 'COUNTRY', 'REGION', 'TIMEZONE',
    'LAST_ORDER_VALUE', 'TOTAL_ORDERS_VALUE', 'AVG_ORDER_VALUE',
    'LATEST_CLICK_CLIENT_TYPE', 'LATEST_CLICK_CLIENT_OS_FAMILY', 'LATEST_CLICK_CLIENT_NAME',
    'FIRST_ACTIVE_TS_DAYS', 'FIRST_UTM_SOURCE', 'FIRST_UTM_CONTENT', 'FIRST_UTM_CAMPAIGN'
]

# Variation features: creative features
variation_cols = [
    'VARIATION_ID', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'category'
]

print("Recipient feature columns:")
print(recipient_cols)
print(f"\nVariation feature columns:")
print(variation_cols)
print(f"\n✓ Total cols in df: {len(df.columns)}, categorized: {len(recipient_cols) + len(variation_cols) + 1}")  # +1 for CLICKED

Recipient feature columns:
['EXPERIMENT_ID', 'RECIPIENT_ID', 'MONTH', 'DAY', 'HOUR', 'CITY', 'COUNTRY', 'REGION', 'TIMEZONE', 'LAST_ORDER_VALUE', 'TOTAL_ORDERS_VALUE', 'AVG_ORDER_VALUE', 'LATEST_CLICK_CLIENT_TYPE', 'LATEST_CLICK_CLIENT_OS_FAMILY', 'LATEST_CLICK_CLIENT_NAME', 'FIRST_ACTIVE_TS_DAYS', 'FIRST_UTM_SOURCE', 'FIRST_UTM_CONTENT', 'FIRST_UTM_CAMPAIGN']

Variation feature columns:
['VARIATION_ID', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'category']

✓ Total cols in df: 32, categorized: 32


## 5. Recall@K Evaluation Functions
Functions for evaluating recommendation model performance using recall@k metric

In [107]:
def enrich_with_unseen_variations(df, recipient_cols, variation_cols, verbose=False):
    """
    For each (recipient, experiment) pair in the dataset, add rows for all variations
    they didn't see in that experiment. Each new row duplicates recipient features
    but uses the variation features from the unseen variation.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Original dataset with one row per (recipient, experiment, variation) observation
    recipient_cols : list
        List of column names that are recipient-specific features
    variation_cols : list
        List of column names that are variation-specific features (including VARIATION_ID)
    verbose : bool
        If True, print progress information
    
    Returns:
    --------
    pd.DataFrame
        Enriched dataset with additional rows for unseen variations, with CLICKED=0 for new rows
    """
    if verbose:
        print("Starting enrichment process...")
        print(f"Initial dataset size: {len(df):,} rows")
    
    # Create a lookup table for variation features
    variation_features = df[variation_cols].drop_duplicates(subset=['VARIATION_ID'])
    if verbose:
        print(f"Unique variations found: {len(variation_features)}")
    
    # Get all variations per experiment
    experiment_variations = df.groupby('EXPERIMENT_ID')['VARIATION_ID'].apply(set).to_dict()
    
    # Process each experiment separately
    enriched_dfs = []
    
    for exp_id, variations_in_exp in experiment_variations.items():
        # Get all rows for this experiment
        exp_df = df[df['EXPERIMENT_ID'] == exp_id].copy()
        
        # Get unique recipients in this experiment
        recipients = exp_df['RECIPIENT_ID'].unique()
        
        # Get variation features for this experiment only
        exp_variations = variation_features[variation_features['VARIATION_ID'].isin(variations_in_exp)]
        
        new_rows = []
        
        for recipient_id in recipients:
            # Get the recipient's existing row(s) in this experiment
            recipient_rows = exp_df[exp_df['RECIPIENT_ID'] == recipient_id]
            
            # Get the variation(s) this recipient saw
            seen_variations = set(recipient_rows['VARIATION_ID'].values)
            
            # Get unseen variations
            unseen_variations = variations_in_exp - seen_variations
            
            if len(unseen_variations) > 0:
                # Take one representative row for recipient features
                recipient_features = recipient_rows[recipient_cols].iloc[0]
                
                # Create a row for each unseen variation
                for unseen_var_id in unseen_variations:
                    var_features = exp_variations[exp_variations['VARIATION_ID'] == unseen_var_id]
                    
                    if len(var_features) > 0:
                        # Combine recipient and variation features
                        new_row = pd.concat([
                            recipient_features,
                            var_features.iloc[0],
                            pd.Series({'CLICKED': 0})
                        ])
                        new_rows.append(new_row)
        
        if new_rows:
            new_rows_df = pd.DataFrame(new_rows)
            enriched_dfs.append(pd.concat([exp_df, new_rows_df], ignore_index=True))
        else:
            enriched_dfs.append(exp_df)
        
        if verbose:
            print(f"  Experiment {exp_id[:8]}...: {len(exp_df):,} → {len(enriched_dfs[-1]):,} rows")
    
    # Combine all experiments
    result_df = pd.concat(enriched_dfs, ignore_index=True)
    
    if verbose:
        print(f"\n✓ Enrichment complete!")
        print(f"  Final dataset size: {len(result_df):,} rows")
        print(f"  Added: {len(result_df) - len(df):,} rows")
    
    return result_df


def recall_at_k(X, y, model, k, recipient_cols, variation_cols, categorical_features=None, verbose=False):
    """
    Calculate recall@k for a recommendation system where each recipient clicks on exactly one variation
    per experiment. The function enriches the data with unseen variations, predicts scores,
    and measures what proportion of recipients have their clicked variation in the top-k recommendations.
    
    Since each recipient clicks exactly one variation:
    - Recall for a recipient = 1 if clicked variation is in top-k, else 0
    - Overall recall@k = average across all recipients (proportion who were "recalled")
    
    Parameters:
    -----------
    X : pd.DataFrame
        Features (without target)
    y : pd.Series
        Target variable (CLICKED)
    model : trained model
        Model with predict_proba or predict method
    k : int
        Number of top recommendations to consider
    recipient_cols : list
        List of recipient feature column names
    variation_cols : list
        List of variation feature column names
    categorical_features : list, optional
        List of categorical feature names to handle NaN values
    verbose : bool
        If True, print detailed progress information
    
    Returns:
    --------
    float
        Recall@k score (proportion of recipients whose clicked variation was in top-k)
    dict
        Dictionary with detailed metrics
    """
    # Concatenate X and y
    df_eval = X.copy()
    df_eval['CLICKED'] = y.values
    
    # Enrich with unseen variations
    df_enriched = enrich_with_unseen_variations(df_eval, recipient_cols, variation_cols, verbose=verbose)
    
    # Handle NaN values in categorical features for prediction
    if categorical_features is not None:
        for col in categorical_features:
            if col in df_enriched.columns:
                df_enriched[col] = df_enriched[col].fillna('missing').astype(str)
    
    # Split back into features and target
    y_hat = df_enriched['CLICKED']
    X_hat = df_enriched.drop(columns=['CLICKED'])
    
    # Predict probabilities
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_hat)[:, 1]
    else:
        y_pred_proba = model.predict(X_hat)
    
    df_enriched['prediction'] = y_pred_proba
    
    # Calculate recall@k for each (recipient, experiment) pair
    recalls = []
    
    for (exp_id, recipient_id), group in df_enriched.groupby(['EXPERIMENT_ID', 'RECIPIENT_ID']):
        # Sort by prediction score (descending)
        group_sorted = group.sort_values('prediction', ascending=False)
        
        # Get top-k recommendations
        top_k = group_sorted.head(k)
        
        # Check if the clicked variation is in top-k
        # Since each recipient clicks on exactly ONE variation, recall is 1 if it's in top-k, else 0
        clicks_in_top_k = top_k['CLICKED'].sum()
        if clicks_in_top_k > 0:
            recall = 1.0
        else:
            recall = 0.0
        recalls.append(recall)
    
    # Average recall@k across all recipients
    mean_recall_at_k = np.mean(recalls)
    
    # Additional metrics
    metrics = {
        'recall_at_k': mean_recall_at_k,
        'k': k,
        'total_recipients': len(recalls),
        'recipients_recalled': sum(recalls),
        'total_enriched_rows': len(df_enriched),
        'original_rows': len(X),
        'enrichment_ratio': len(df_enriched) / len(X)
    }
    
    return mean_recall_at_k, metrics

print("✓ Evaluation functions defined successfully!")


✓ Evaluation functions defined successfully!


## 6. CatBoost Model Training
Training a CatBoost classifier with Logloss to predict click probability

In [108]:
# Import CatBoost
from catboost import CatBoostClassifier, Pool

# Identify categorical and numerical features
# All string/object columns and IDs are categorical
categorical_features = [
    'EXPERIMENT_ID', 'VARIATION_ID', 'RECIPIENT_ID',
    'CITY', 'COUNTRY', 'REGION', 'TIMEZONE',
    'LATEST_CLICK_CLIENT_TYPE', 'LATEST_CLICK_CLIENT_OS_FAMILY', 'LATEST_CLICK_CLIENT_NAME',
    'FIRST_UTM_SOURCE', 'FIRST_UTM_CONTENT', 'FIRST_UTM_CAMPAIGN',
    'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'category'
]

# Numerical features are the remaining ones
numerical_features = [col for col in X_train.columns if col not in categorical_features]

print(f"✓ Feature categorization complete")
print(f"\nCategorical features ({len(categorical_features)}):")
for feat in categorical_features:
    print(f"  - {feat}")
    
print(f"\nNumerical features ({len(numerical_features)}):")
for feat in numerical_features:
    print(f"  - {feat}")

✓ Feature categorization complete

Categorical features (24):
  - EXPERIMENT_ID
  - VARIATION_ID
  - RECIPIENT_ID
  - CITY
  - COUNTRY
  - REGION
  - TIMEZONE
  - LATEST_CLICK_CLIENT_TYPE
  - LATEST_CLICK_CLIENT_OS_FAMILY
  - LATEST_CLICK_CLIENT_NAME
  - FIRST_UTM_SOURCE
  - FIRST_UTM_CONTENT
  - FIRST_UTM_CAMPAIGN
  - Q1
  - Q2
  - Q3
  - Q4
  - Q5
  - Q6
  - Q7
  - Q8
  - Q9
  - Q10
  - category

Numerical features (7):
  - MONTH
  - DAY
  - HOUR
  - LAST_ORDER_VALUE
  - TOTAL_ORDERS_VALUE
  - AVG_ORDER_VALUE
  - FIRST_ACTIVE_TS_DAYS


In [110]:
# Handle NaN values in categorical features by converting to string
# CatBoost requires categorical features to be string or int, not NaN

X_train_processed = X_train.copy()
X_val_processed = X_val.copy()
X_test_processed = X_test.copy()

# Convert NaN to string 'missing' for categorical features
for col in categorical_features:
    if col in X_train_processed.columns:
        X_train_processed[col] = X_train_processed[col].fillna('missing').astype(str)
        X_val_processed[col] = X_val_processed[col].fillna('missing').astype(str)
        X_test_processed[col] = X_test_processed[col].fillna('missing').astype(str)

print("✓ Processed NaN values in categorical features")
print(f"  Converted {len(categorical_features)} categorical columns")

# Create CatBoost Pools for efficient training
train_pool = Pool(
    data=X_train_processed,
    label=y_train,
    cat_features=categorical_features
)

val_pool = Pool(
    data=X_val_processed,
    label=y_val,
    cat_features=categorical_features
)

print("\n✓ CatBoost Pools created successfully!")
print(f"  Train pool: {train_pool.num_row()} rows, {train_pool.num_col()} features")
print(f"  Val pool: {val_pool.num_row()} rows, {val_pool.num_col()} features")

✓ Processed NaN values in categorical features
  Converted 24 categorical columns

✓ CatBoost Pools created successfully!
  Train pool: 76818 rows, 31 features
  Val pool: 8641 rows, 31 features


In [119]:
# Initialize CatBoost model
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=100,
    early_stopping_rounds=50,
    use_best_model=True
)

print("🚀 Starting CatBoost training...")
print("=" * 60)

# Train the model
model.fit(
    train_pool,
    eval_set=val_pool,
    plot=False
)

print("\n" + "=" * 60)
print("✓ Training complete!")
print(f"Best iteration: {model.best_iteration_}")
print(f"Best validation score: {model.best_score_['validation']['AUC']:.4f}")

🚀 Starting CatBoost training...
0:	test: 0.6306449	best: 0.6306449 (0)	total: 42ms	remaining: 41.9s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7950325419
bestIteration = 43

Shrink model to first 44 iterations.

✓ Training complete!
Best iteration: 43
Best validation score: 0.7950


# Evaluate on validation set
print("📊 Evaluating on VALIDATION set...")
print("=" * 60)

# Test multiple k values
k_values = [1, 3, 5]
val_results = {}

for k in k_values:
    recall, metrics = recall_at_k(
        X_val_processed, y_val, model, k,
        recipient_cols, variation_cols,
        categorical_features=categorical_features,
        verbose=False
    )
    val_results[k] = metrics
    print(f"\nRecall@{k}:")
    print(f"  Score: {recall:.4f} ({recall*100:.2f}%)")
    print(f"  Recipients recalled: {metrics['recipients_recalled']:.0f} / {metrics['total_recipients']}")

print("\n" + "=" * 60)

In [115]:
# Evaluate on test set
print("📊 Evaluating on TEST set...")
print("=" * 60)

test_results = {}
k_values = [1, 3, 5, 7]
for k in k_values:
    recall, metrics = recall_at_k(
        X_test_processed, y_test, model, k,
        recipient_cols, variation_cols,
        categorical_features=categorical_features,
        verbose=False
    )
    test_results[k] = metrics
    print(f"\nRecall@{k}:")
    print(f"  Score: {recall:.4f} ({recall*100:.2f}%)")
    print(f"  Recipients recalled: {metrics['recipients_recalled']:.0f} / {metrics['total_recipients']}")

print("\n" + "=" * 60)

📊 Evaluating on TEST set...

Recall@1:
  Score: 0.5043 (50.43%)
  Recipients recalled: 2370 / 4700

Recall@3:
  Score: 0.5043 (50.43%)
  Recipients recalled: 2370 / 4700

Recall@5:
  Score: 0.5043 (50.43%)
  Recipients recalled: 2370 / 4700

Recall@7:
  Score: 0.5043 (50.43%)
  Recipients recalled: 2370 / 4700



In [ ]:
# Summary comparison
print("📈 RESULTS SUMMARY")
print("=" * 60)
print(f"\n{'Metric':<15} {'Validation':<15} {'Test':<15}")
print("-" * 45)

for k in k_values:
    val_score = val_results[k]['recall_at_k']
    test_score = test_results[k]['recall_at_k']
    print(f"Recall@{k:<10} {val_score:<15.4f} {test_score:<15.4f}")

print("\n" + "=" * 60)
print("✓ Model training and evaluation complete!")